In [9]:
import ee
import geemap
import requests
import numpy as np
from io import BytesIO
from PIL import Image
import matplotlib.pyplot as plt

In [10]:
ee.Authenticate()
ee.Initialize(project="deforestationsentinel2")

In [11]:
# Region of interest: San Vicente del Caguán and La Macarena municipalities
roi = ee.Geometry.BBox(-73, 2.8, -72, 2.0)
#roi = ee.Geometry.BBox(-74.2, 2, -73.9, 2.3) # Test ROI

# Date range for the analysis
start_date = '2022-01-01'
end_date = '2022-12-31'


In [12]:
# ------------------------------
# 1. Sentinel-2:
# ------------------------------
s2_bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B11', 'B12']
sentinel = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(roi) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .filterDate(start_date, end_date) \
    .select(s2_bands)  # Select only the bands of interest


# ------------------------------
# 2. Dynamic World:
# ------------------------------

dynamic = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1') \
    .filterBounds(roi) \
    .filterDate(start_date, end_date)  \
    .select('label')  # Keep only the class label


In [13]:
def collection_info(collection):
    collection_size = collection.size().getInfo()
    first = collection.first()
    collection_id = collection.get('system:id').getInfo() or "Unnamed Collection"
    if collection_size == 0:
        print("No images found for the specified criteria. Please adjust the date range or location.")
        return None
    else:
        print(f"Found {collection_size} images in {collection_id}.")
        # Get info from the first image
        first_info = first.getInfo()
        print("General info about the first image:")
        print(f"  ID: {first_info.get('id', 'N/A')}")
        print(f"  Bands: {[b['id'] for b in first_info['bands']]}")
        print(f"  Properties: {list(first_info['properties'].keys())[:8]} ...")
        print(f"  Date: {first_info['properties'].get('system:time_start', 'N/A')}")
        print("")

collection_info(sentinel)
collection_info(dynamic)

Found 26 images in COPERNICUS/S2_SR_HARMONIZED.
General info about the first image:
  ID: COPERNICUS/S2_SR_HARMONIZED/20220105T151711_20220105T151705_T18NYJ
  Bands: ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B11', 'B12']
  Properties: ['DATATAKE_IDENTIFIER', 'AOT_RETRIEVAL_ACCURACY', 'SPACECRAFT_NAME', 'SATURATED_DEFECTIVE_PIXEL_PERCENTAGE', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A', 'CLOUD_SHADOW_PERCENTAGE', 'MEAN_SOLAR_AZIMUTH_ANGLE', 'system:footprint'] ...
  Date: 1641396134047

Found 78 images in GOOGLE/DYNAMICWORLD/V1.
General info about the first image:
  ID: GOOGLE/DYNAMICWORLD/V1/20220105T151711_20220105T151705_T18NYH
  Bands: ['label']
  Properties: ['system:time_start', 'dynamicworld_algorithm_version', 'qa_algorithm_version', 'system:footprint', 'system:asset_size', 'system:index'] ...
  Date: 1641396148704



In [14]:
dw_class_names = [
    'water',
    'trees',
    'grass',
    'flooded_vegetation',
    'crops',
    'shrub_and_scrub',
    'built',
    'bare',
    'snow_and_ice',
]

VIS_PALETTE = [
    '419bdf',
    '397d49',
    '88b053',
    '7a87c6',
    'e49635',
    'dfc35a',
    'c4281b',
    'a59b8f',
    'b39fe1',
]

In [ ]:
# =================================================================

roi_area = roi.area(maxError=1).divide(1e6).getInfo()  # Area in km²
print(f"ROI area: {roi_area:.2f} km²")

# Define the scale (resolution in meters/pixel).
# For Sentinel-2 true-color bands (B2, B3, B4), the native resolution is 10 meters.
scale = 10

# Get the bounding box of our region.
info = roi.getInfo()
bounds = roi.bounds()

# The coordinates are returned as a list of lists of [lon, lat] pairs.
# For a rectangle, it's [[p1, p2, p3, p4, p1]]. We need the corners.
coords = ee.List(bounds.coordinates().get(0))

# Get the lower-left and upper-right corners.
ll = ee.Geometry.Point(coords.get(0)) # Lower-Left
lr = ee.Geometry.Point(coords.get(1)) # Lower-Right
ul = ee.Geometry.Point(coords.get(3)) # Upper-Left

# Calculate width and height in meters. These are server-side objects.
width_m_server = ll.distance(lr)
height_m_server = ll.distance(ul)

# Get the numbers from the server to the client.
width_m = width_m_server.getInfo()
height_m = height_m_server.getInfo()

# Calculate pixel dimensions.
width_pixels = round(width_m / scale)
height_pixels = round(height_m / scale)

print(f"ROI Analysis Scale: {scale} meters/pixel")
print("\nDimensions in Meters:")
# Use f-string formatting to add commas for readability
print(f"  - Width:  {width_m:,.2f} meters")
print(f"  - Height: {height_m:,.2f} meters")

print("\nDimensions in Pixels (at the specified scale):")
print(f"  - Width:  {width_pixels:,} pixels")
print(f"  - Height: {height_pixels:,} pixels")

ROI area: 9882.72 km²
ROI Analysis Scale: 10 meters/pixel

Dimensions in Meters:
  - Width:  111,252.13 meters
  - Height: 88,460.00 meters

Dimensions in Pixels (at the specified scale):
  - Width:  11,125 pixels
  - Height: 8,846 pixels


In [19]:
# =================================================================
# Visualize the Image
# =================================================================
vis_params_true_color = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 3000,
    'gamma': 1.4,
}


Map = geemap.Map()
Map.centerObject(roi, 10)


# Add the ROI boundary to the map
Map.addLayer(
    roi,
    {'color': 'orange', 'fillColor': '00000000'}, # Style as a yellow outline
    'ROI Boundary'
)

Map.addLayerControl()

display(Map)

Map(center=[2.399991412723134, -72.50000000000001], controls=(WidgetControl(options=['position', 'transparent_…